<a href="https://colab.research.google.com/github/rrfsantos/Desafios-NLP/blob/main/Desafio-NLP--NER/insert_label_iob_sent_convert_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip uninstall -y spacy==2.2.4
!python -m pip install pip setuptools wheel
!python -m pip install spacy[transformers,lookups]
!python -m spacy download en_core_web_sm  ### o colab não suportou executar a conversão com en_core_web_md e en_core_web_lg

In [2]:
import pandas as pd
import warnings
import glob
import spacy
import tqdm
from os import system
from os.path import join, basename
from tqdm import tqdm_notebook

warnings.filterwarnings("ignore")

In [3]:
!python -m spacy info

2021-04-29 18:36:26.638234: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

============================== Info about spaCy ==============================

spaCy version    3.0.6                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.10                        
Pipelines        en_core_web_sm (3.0.0)        



In [4]:
from google.colab import drive
drive.mount('/content/drive')

import os
workdir_path = '/content/drive/My Drive/desafio 2/'
os.chdir(workdir_path)

Mounted at /content/drive


In [5]:
folder_paths = glob.glob('NERdata/*')

##### Ler arquivos IOB e incluir o nome da classe à tag

In [6]:
# O nome do tipo de entidade para cada dataset
DATASET_NAME_2_ENTITY_TYPE=\
{
    'BC2GM':'DRUG_PROTEIN',
    'BC4CHEMD':'CHEMICAL',
    'BC5CDR-disease':'DISEASE',
    'BC5CDR-chem':'CHEMICAL',
    'JNLPBA':'DRUG_PROTEIN',
    'linnaeus':'SPECIES',
    'NCBI-disease':'DISEASE',
    's800':'SPECIES',
}

In [7]:
def convert_to_iob_format(input_file, output_file, entity_type):
    # Abrimos o arquivo de anotação base 
    with open(input_file,'r') as file:
        lines = file.readlines()

    sents=[] # Lista de sentenças 
    sent=[] # Uma sentença
    
    # Iteramos ao longo do documento 
    for line in lines:
        line=line.strip()
        
        if line:
              # Para cada linha removemos o quebre de linha e separamos a palavra do tag
              word,tag=line.replace('\n','').split('\t')

              if (word != '') and (tag != ''):              
                # Anotamos o complemento I -> I->ENTITY e B -> B-entity
                if tag in ['I','B']:
                    tag=f'{tag}-{entity_type}'

                # Armazenamos a palavra e o tag no formato IOB
                    sent.append(f'{word}'+'|''|'+f'{tag}')
        else:
            # se a linha estiver vacia então indica o final da linha. 
            # Se armazena a sentença e se limpa "sent" par coletar a seguinte linha
            sents.append(sent)
            sent=[]

    # Juntamos todas as sentenças por espaço ' ' e todas as linhas por quebre de linha '\n'
    sents_content = '\n'.join([' '.join(sent) for sent in sents])

    # Salvamos o formato IOB
    with open(output_file,'w+') as file:
        file.write(sents_content)

In [8]:
for folder_path in tqdm_notebook(folder_paths, total=len(folder_paths)):
    
    dataset_name = basename(folder_path)
    
    entity_type = DATASET_NAME_2_ENTITY_TYPE[dataset_name]
    
    for input_file in glob.glob(join(folder_path,'*.tsv')):

        iob_file = input_file.replace('.tsv','.iob')
        
        convert_to_iob_format(input_file, iob_file, entity_type)

        folder_spacy = 'SENTdata/' + dataset_name
           
        #system(f'python -m spacy convert -c iob -s -n 10 -b en_core_web_sm {iob_file} {folder_spacy}')

##### Converter arquivos em formato IOB para o formato binário .spacy

In [9]:
for folder_path in tqdm_notebook(folder_paths, total=len(folder_paths)):

  dataset_name = basename(folder_path)

  for input_file in glob.glob(join(folder_path,'*.iob')):

    folder_spacy = 'SENTdata/' + dataset_name

    system(f'python -m spacy convert -c iob -s -n 10 {input_file} {folder_spacy}')